In [1]:
#pylab inline
import numpy
np = numpy
import matplotlib
from matplotlib import pylab, mlab, pyplot
plt = pyplot
from IPython.core.pylabtools import figsize, getfigs

import asyncio

import subprocess
import requests
import json
import pprint
import datetime
import time
import re
from tqdm.notebook import tqdm
import pandas as pd
import collections
import operator
import itertools
import os
import typing

import json
import hjson

amap = lambda *args, **kwargs: np.array(list(map(*args)), **kwargs)

In [2]:
from llama_cpp import Llama, LlamaGrammar
from llama_cpp.llama_speculative import LlamaDraftModel, LlamaPromptLookupDecoding

llm = Llama(
    model_path='/Users/abra/.cache/lm-studio/models/lmstudio-community/Llama-3.3-70B-Instruct-GGUF/Llama-3.3-70B-Instruct-Q8_0-00001-of-00002.gguf',
    # model_path='/Users/abra/.cache/lm-studio/models/lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF/Meta-Llama-3.1-8B-Instruct-Q4_K_M.gguf',
    # model_path='/Users/abra/.cache/lm-studio/models/lmstudio-community/Llama-3.3-70B-Instruct-GGUF/Llama-3.3-70B-Instruct-Q4_K_M.gguf',
    # model_path='/Users/abra/.cache/lm-studio/models/lmstudio-community/Meta-Llama-3.1-70B-Instruct-GGUF/Meta-Llama-3.1-70B-Instruct-Q8_0-00001-of-00002.gguf',
    logits_all=True,
    n_ctx=1024 * 4,
    n_gpu_layers=-1,
    # draft_model=LlamaPromptLookupDecoding(),
)


llama_load_model_from_file: using device Metal (Apple M3 Max) - 98303 MiB free
llama_model_loader: additional 1 GGUFs metadata loaded.
llama_model_loader: loaded meta data with 39 key-value pairs and 724 tensors from /Users/abra/.cache/lm-studio/models/lmstudio-community/Llama-3.3-70B-Instruct-GGUF/Llama-3.3-70B-Instruct-Q8_0-00001-of-00002.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Llama 3.3 70B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = Llama-3.3
llama_model_l

In [3]:
TOKEN_BEGIN_OF_TEXT = llm.tokenize(b'<|begin_of_text|>', add_bos=False, special=True)
TOKEN_END_OF_TURN = llm.tokenize(b'<|eot_id|>', add_bos=False, special=True)
TOKEN_END_OF_MESSAGE = llm.tokenize(b'<|eom_id|>', add_bos=False, special=True)
TOKEN_START_TURN_HEADER = llm.tokenize(b'<|start_header_id|>', add_bos=False, special=True)
TOKEN_END_TURN_HEADER = llm.tokenize(b'<|end_header_id|>', add_bos=False, special=True)

def str_to_tokens(s):
    return llm.tokenize(
        s.encode(), 
        add_bos=False, 
        special=False,
    )

In [4]:
last_input_ids = None
last_scores = None

def global_logging_logit_processor(
    input_ids: np.typing.NDArray[np.intc], scores: np.typing.NDArray[np.single],
) -> np.typing.NDArray[np.single]:
    global last_input_ids, last_scores
    last_input_ids = np.array(input_ids)
    last_scores = np.array(scores)

    return scores

json_schema_str = '''

{
  "type": "object",
  "properties": {
    "german_saying": { "type": "string" },
#    "literal_english_translation": { "type": "string" },
#    "meaning": { "type": "string" },
  },
  "required": [
    "german_saying",
#    "literal_english_translation",
#    "meaning",
  ]
}

'''

json_schema_str = json.dumps(hjson.loads(json_schema_str))

tokens = (
    TOKEN_BEGIN_OF_TEXT + 

    TOKEN_START_TURN_HEADER + str_to_tokens('system') + TOKEN_END_TURN_HEADER + str_to_tokens('\n\n') +
    str_to_tokens('''
Reply STRICTLY conforming to this JSON schema.
    '''.strip()) +
    str_to_tokens('\n') +
    str_to_tokens(json_schema_str) + 
    str_to_tokens('\n') +
    str_to_tokens('''
Do NOT output anything else besides a single JSON object.
    '''.strip()) +
    TOKEN_END_OF_TURN + 
    
    TOKEN_START_TURN_HEADER + str_to_tokens('user') + TOKEN_END_TURN_HEADER + str_to_tokens('\n\n') +
    str_to_tokens('''
Tell me a German saying.
    '''.strip()) +
    TOKEN_END_OF_TURN + 

    TOKEN_START_TURN_HEADER + str_to_tokens('assistant') + TOKEN_END_TURN_HEADER + str_to_tokens('\n\n') +
    str_to_tokens('''
{"german_saying": "
    '''.strip())
)

# # llm.reset()
# llm.set_seed(0)
# generator = llm.generate(
#     tokens=tokens,
#     logits_processor=[],
#     # grammar=LlamaGrammar.from_json_schema(json_schema_str),
# )

# print(llm.detokenize(tokens, prev_tokens=None, special=True).decode(), end='')

# shown_tokens = list(tokens)
# unshown_tokens = []

# for new_token in generator:
#     if new_token is None or new_token == TOKEN_END_OF_TURN[0]:
#         break

#     unshown_tokens.append(new_token)

#     bytes_to_show = llm.detokenize(unshown_tokens, prev_tokens=shown_tokens, special=True)

#     try:
#         string_to_show = bytes_to_show.decode()
#     except UnicodeDecodeError as e:
#         # spanish inverted question mark
#         print('¿', end='')
#         continue
    
#     shown_tokens.extend(unshown_tokens)
#     unshown_tokens = []
    
#     print(string_to_show, end='')

In [ ]:
generator = llm.generate(
    tokens=tokens,
    logits_processor=[global_logging_logit_processor],
    # grammar=LlamaGrammar.from_json_schema(json_schema_str),
)
next_token = next(generator)
llm.detokenize([next_token], prev_tokens=tokens, special=True)

In [7]:
llm.reset()
llm.eval(tokens)

In [8]:
prompt_fed_state = llm.save_state()

Llama.save_state: saving llama state
Llama.save_state: got state size: 68941001
Llama.save_state: allocated state
Llama.save_state: copied llama state: 68941001
Llama.save_state: saving 68941001 bytes of llama state


In [9]:
import sortedcontainers

In [10]:
from dataclasses import dataclass
@dataclass
class State:
    acc_probability: float
    new_tokens: list[int]
    llm_state_before_last_token: typing.Any
    llm_state_after_last_token: typing.Any

    def __lt__(self, o):
        return self.acc_probability < o.acc_probability


In [73]:
finished_states = sortedcontainers.SortedList()

states = sortedcontainers.SortedList([
    State(1.0, [], None, prompt_fed_state),
])

In [72]:
for _ in tqdm(range(10000)):
    state = states.pop(-1)

    if state.llm_state_after_last_token is not None:
        llm.load_state(state.llm_state_after_last_token)
        llm_state = state.llm_state_after_last_token
    else:
        llm.load_state(state.llm_state_before_last_token)
        llm.eval(state.new_tokens[-1:])
        llm_state = llm.save_state()
    
    last_scores = None
    next_token = llm.sample(
        logits_processor=[global_logging_logit_processor]
    )

    logits = np.array(last_scores)
    logits -= np.max(logits)
    exp_logits = np.exp(logits)
    probs = exp_logits / np.sum(exp_logits)

    top_indices = np.argsort(probs)[-1000:]

    for next_token in top_indices:
        new_prob = state.acc_probability * probs[next_token]

        new_state = State(
            acc_probability=new_prob,
            new_tokens=state.new_tokens + [next_token],
            llm_state_before_last_token=llm_state,
            llm_state_after_last_token=None,
        )
        
        if next_token == TOKEN_END_OF_TURN[0]:
            finished_states.add(new_state)
        else:
            states.add(new_state)

  0%|          | 0/10000 [00:00<?, ?it/s]

Llama.save_state: saving llama state
Llama.save_state: got state size: 27713425
Llama.save_state: allocated state
Llama.save_state: copied llama state: 27713425
Llama.save_state: saving 27713425 bytes of llama state
Llama.save_state: saving llama state
Llama.save_state: got state size: 28041117
Llama.save_state: allocated state
Llama.save_state: copied llama state: 28041117
Llama.save_state: saving 28041117 bytes of llama state
Llama.save_state: saving llama state
Llama.save_state: got state size: 28368809
Llama.save_state: allocated state
Llama.save_state: copied llama state: 28368809
Llama.save_state: saving 28368809 bytes of llama state
Llama.save_state: saving llama state
Llama.save_state: got state size: 28696501
Llama.save_state: allocated state
Llama.save_state: copied llama state: 28696501
Llama.save_state: saving 28696501 bytes of llama state
Llama.save_state: saving llama state
Llama.save_state: got state size: 29024193
Llama.save_state: allocated state
Llama.save_state: copi

In [70]:
print('finished')
for i in finished_states[::-1]:
    print(f'{i.acc_probability:%}', '->', llm.detokenize(i.new_tokens[:-1], prev_tokens=None, special=True).decode())

print()
print('in progress')

for i in states[-10:][::-1]:
    print(f'{i.acc_probability:%}', '->', llm.detokenize(i.new_tokens, prev_tokens=None, special=True).decode())

finished
48.797053% -> Morgenstund hat Gold im Mund"}
38.437265% -> Aller Anfang ist schwer"}
4.872750% -> Morgenstunde hat Gold im Munde"}
0.737206% -> Alle guten Dinge sind drei"}
0.568172% -> Wenn das Leben dir Zitronen gibt, mache Limonade daraus"}
0.372180% -> Aufgeschoben ist nicht aufgehoben"}
0.333183% -> Morgensstund hat Gold im Mund"}
0.292641% -> Wenn das Leben dir Zitronen gibt, mache Limonade"}
0.288285% -> Wenn man nicht wandert, kommt man nicht weiter."}
0.282875% -> Nach dem Regen kommt die Sonne"}
0.226366% -> Wenn man nicht wandert, kommt man nicht weiter"}
0.119454% -> Ich verstehe nur Bahnhof"}
0.103320% -> Wenn man nicht wandert, kommt man nicht vorwärts"}
0.083417% -> Alle Wege führen nach Rom"}
0.082698% -> Wenn die Katze aus dem Haus ist, tanzen die M\u00e4use"}
0.076038% -> Wer anderen eine Grube gr\u00e4bt, f\u00e4llt selbst hinein"}
0.065530% -> Wenn man nicht wandert, kommt man nicht zum Ziel"}
0.064269% -> Alle guten Dinge sind drei."}
0.062211% -> Übung ma

In [63]:
import unicodedata
for i in s:
    print(i, unicodedata.name(i))

W LATIN CAPITAL LETTER W
e LATIN SMALL LETTER E
n LATIN SMALL LETTER N
n LATIN SMALL LETTER N
  SPACE
d LATIN SMALL LETTER D
i LATIN SMALL LETTER I
e LATIN SMALL LETTER E
  SPACE
K LATIN CAPITAL LETTER K
a LATIN SMALL LETTER A
t LATIN SMALL LETTER T
z LATIN SMALL LETTER Z
e LATIN SMALL LETTER E
  SPACE
a LATIN SMALL LETTER A
u LATIN SMALL LETTER U
s LATIN SMALL LETTER S
  SPACE
d LATIN SMALL LETTER D
e LATIN SMALL LETTER E
m LATIN SMALL LETTER M
  SPACE
H LATIN CAPITAL LETTER H
a LATIN SMALL LETTER A
u LATIN SMALL LETTER U
s LATIN SMALL LETTER S
  SPACE
i LATIN SMALL LETTER I
s LATIN SMALL LETTER S
t LATIN SMALL LETTER T
, COMMA
  SPACE
t LATIN SMALL LETTER T
a LATIN SMALL LETTER A
n LATIN SMALL LETTER N
z LATIN SMALL LETTER Z
e LATIN SMALL LETTER E
n LATIN SMALL LETTER N
  SPACE
d LATIN SMALL LETTER D
i LATIN SMALL LETTER I
e LATIN SMALL LETTER E
  SPACE
M LATIN CAPITAL LETTER M
\ REVERSE SOLIDUS
u LATIN SMALL LETTER U
0 DIGIT ZERO
0 DIGIT ZERO
e LATIN SMALL LETTER E
4 DIGIT FOUR
u LA